In [28]:
import sys

from models.DeepDense import DeepDense
from models.TextLSTM import TextLSTM
from models.DNNAttr import DNNAttr
from models.Wide import Wide

from models.WideDeep import WideDeep
from optim.Initializer import KaimingNormal, XavierNormal
from optim.radam import RAdam
from pandas import DataFrame
from preprocessing.Preprocessor import WidePreprocessor, DeepPreprocessor, DeepTextPreprocessor, MultiDeepTextPreprocessor

import numpy as np
import pandas as pd
import torch, os
from torch.utils.data import DataLoader
import time
import torch


wide = Wide(wide_dim=17, output_dim=1)

# init deep_dense model
deep_column_idx = dict()
deep_column_idx['family_pred_gender'] = 0
deep_column_idx['family_pred_age_level'] = 1
deep_column_idx['category'] = 2
deep_column_idx['ott_uv_norm'] = 3
deep_column_idx['category_prefer'] = 4
emb_col_val_dim_tuple = []
emb_col_val_dim_tuple.append(('family_pred_gender', 4, 8))
emb_col_val_dim_tuple.append(('family_pred_age_level', 1024, 12))
emb_col_val_dim_tuple.append(('category', 28, 8))
deepdense = DeepDense(hidden_layers=[32], dropout=[0.2], deep_column_idx=deep_column_idx, embed_input=emb_col_val_dim_tuple, continuous_cols=['ott_uv_norm', 'category_prefer'])

# init transformer model
transformer = DNNAttr()

wide_deep_model = WideDeep(wide=wide, deepdense=deepdense, deeptext=transformer, head_layers=[128])
wide_deep_model.load_state_dict(torch.load('log/05-09_14.14/sug_saved_model.pt'))

for name, param in wide_deep_model.named_parameters():
    print(name, '        ', param.size())

deep dense 维度：32


RuntimeError: Error(s) in loading state_dict for WideDeep:
	size mismatch for deeptext.embedding.weight: copying a param with shape torch.Size([142710, 32]) from checkpoint, the shape in current model is torch.Size([180000, 32]).
	size mismatch for prefix_embedding.weight: copying a param with shape torch.Size([85175, 32]) from checkpoint, the shape in current model is torch.Size([105000, 32]).

In [ ]:
wide_linear_weight=''
wide_linear_bias=''
emb_layer_category=''
emb_layer_family_pred_age_level=''
emb_layer_family_pred_gender=''
dense_layer_0_w=''
dense_layer_0_bias=''
dense_layer_1_w=''
dense_layer_1_bias=''
query_embedding=''
query_embedding_fc_w=''
query_embedding_fc_bias=''
prefix_embedding=''
head_layer_0_w=''
head_layer_0_bias=''
head_layer_1_w=''
head_layer_1_bias=''
head_out_w=''
head_out_bias=''
for name, param in wide_deep_model.named_parameters():
    
    if 'wide.wide_linear.weight' in name:
        wide_linear_weight = np.round(param.detach().numpy(), 3)
    
    if 'wide.wide_linear.bias' in name:
        wide_linear_bias = np.round(param.detach().numpy(), 3)
    
    # deepdense
    if 'deepdense.embed_layers_dic.emb_layer_category.weight' in name:
        emb_layer_category = np.round(param.detach().numpy(), 3)
    
    if 'deepdense.embed_layers_dic.emb_layer_family_pred_age_level.weight' in name:
        emb_layer_family_pred_age_level = np.round(param.detach().numpy(), 3)
        
    if 'deepdense.embed_layers_dic.emb_layer_family_pred_gender.weight' in name:
        emb_layer_family_pred_gender = np.round(param.detach().numpy(), 3)
    
    if 'deepdense.dense_sequential.dense_layer_0.0.weight' in name:
        dense_layer_0_w = np.round(param.detach().numpy(), 3)
    if 'deepdense.dense_sequential.dense_layer_0.0.bias' in name:
        dense_layer_0_bias = np.round(param.detach().numpy(), 3)
    if 'deepdense.dense_sequential.dense_layer_1.0.weight' in name:
        dense_layer_1_w = np.round(param.detach().numpy(), 3)
    if 'deepdense.dense_sequential.dense_layer_1.0.bias' in name:
        dense_layer_1_bias = np.round(param.detach().numpy(), 3)
    
    
    # text
    if 'deeptext.embedding.weight' in name:
        query_embedding = np.round(param.detach().numpy(), 3)
    if 'deeptext.fc.weight' in name:
        query_embedding_fc_w = np.round(param.detach().numpy(), 3)
    if 'deeptext.fc.bias' in name:
        query_embedding_fc_bias = np.round(param.detach().numpy(), 3)
    
    # prefix
    if 'prefix_embedding.weight' in name:
        prefix_embedding = np.round(param.detach().numpy(), 3)
    if 'deephead.head_layer_0.0.weight' in name:
        head_layer_0_w = np.round(param.detach().numpy(), 3)
    if 'deephead.head_layer_0.0.bias' in name:
        head_layer_0_bias = np.round(param.detach().numpy(), 3)
    
    if 'deephead.head_layer_1.0.weight' in name:
        head_layer_1_w = np.round(param.detach().numpy(), 3)
    if 'deephead.head_layer_1.0.bias' in name:
        head_layer_1_bias = np.round(param.detach().numpy(), 3)
    
    if 'deephead.head_out.weight' in name:
        head_out_w = np.round(param.detach().numpy(), 3)
    
    if 'deephead.head_out.bias' in name:
        head_out_bias = np.round(param.detach().numpy(), 3)
    
np.savez("param.npz", wide_linear_weight=wide_linear_weight,
                        wide_linear_bias=wide_linear_bias,
                        emb_layer_category=emb_layer_category,
                        emb_layer_family_pred_age_level=emb_layer_family_pred_age_level,
                        emb_layer_family_pred_gender=emb_layer_family_pred_gender,
                        dense_layer_0_w=dense_layer_0_w,
                        dense_layer_0_bias=dense_layer_0_bias,
                        dense_layer_1_w=dense_layer_1_w,
                        dense_layer_1_bias=dense_layer_1_bias,
                        query_embedding=query_embedding,
                        query_embedding_fc_w=query_embedding_fc_w,
                        query_embedding_fc_bias=query_embedding_fc_bias,
                        prefix_embedding=prefix_embedding,
                        head_layer_0_w=head_layer_0_w,
                        head_layer_0_bias=head_layer_0_bias,
                        head_layer_1_w=head_layer_1_w,
                        head_layer_1_bias=head_layer_1_bias,
                        head_out_w=head_out_w,
                        head_out_bias=head_out_bias)
    
    
#     if 'emb_layer_family_pred_age_level' in name:
#         emb_layer_family_pred_age_level = np.round(param.detach().numpy(), 3)
        
#         print(name)
#         print(emb_layer_family_pred_age_level)

#     if 'wide.wide_linear.weight' in name:
#         print(wide_linear_weight)
        
        
np_file = np.load("param.npz")
print(np_file['query_embedding'].shape)
print(np_file['query_embedding'])




In [27]:
# [rows, cols] = np_file['dense_layer_1_bias'].shape

print(np_file['head_out_w'])


[[ 0.443  0.212  0.236 -0.163 -0.278 -0.022  0.075  0.231 -0.211 -0.295
  -0.141 -0.128 -0.361  0.392  0.344  0.319 -0.286  0.214 -0.342  0.216
   0.367 -0.185 -0.254 -0.286  0.413 -0.328 -0.269 -0.203 -0.141  0.264
  -0.067 -0.17  -0.112  0.388 -0.353  0.181  0.142  0.29   0.005  0.213
  -0.378  0.38  -0.333  0.382 -0.198 -0.458  0.14   0.408 -0.292 -0.16
  -0.157 -0.16  -0.114  0.455 -0.171 -0.048 -0.278  0.226 -0.146 -0.193
   0.248  0.334  0.126  0.258  0.196  0.1   -0.43   0.181 -0.23   0.135
  -0.354  0.357 -0.153  0.12  -0.207  0.241 -0.26   0.262  0.137  0.271
   0.403  0.207  0.213  0.15   0.042  0.167  0.279 -0.393  0.284  0.036
   0.328  0.233  0.357 -0.146 -0.184  0.035 -0.447  0.201 -0.188  0.643
  -0.118  0.351 -0.231  0.11  -0.248  0.494 -0.294 -0.252  0.284 -0.354
   0.159 -0.083  0.216  0.458  0.133 -0.276 -0.224 -0.14  -0.149  0.284
  -0.197 -0.199  0.26   0.343 -0.461  0.174  0.298  0.283]]
